# Spark + sklearn

---

S.Yu. Papulin (papulin_bmstu@mail.ru)

### Contents

- [Prediction for DataFrame](#Prediction-for-DataFrame)
    - [Using UDF](#Using-UDF)
    - [Using Pandas UDF](#Using-Pandas-UDF)
- [Model Selection](#Model-Selection)

## Prediction for DataFrame

Sklearn related packages

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn import datasets
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error

Spark related packages

In [ ]:
import os
import sys

os.environ["SPARK_HOME"] = "/home/ubuntu/BigData/spark"
os.environ["PYSPARK_PYTHON"] = "/home/ubuntu/ML/anaconda3/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/home/ubuntu/ML/anaconda3/bin/python"

spark_home = os.environ.get("SPARK_HOME")
sys.path.insert(0, os.path.join(spark_home, "python"))
sys.path.insert(0, os.path.join(spark_home, "python/lib/py4j-0.10.7-src.zip"))

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql.types import (
    FloatType,
    ArrayType,
    StringType
)
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, pandas_udf

### Loading dataset

In [ ]:
RANDOM_STATE = 12345

In [ ]:
# Загрузка исходных данных
housing = datasets.fetch_california_housing()

In [ ]:
print(housing["DESCR"])

In [ ]:
X = housing.data[:,:6]
y = housing.target

In [ ]:
CLMS = housing.feature_names[:6] + ["MEDV"]
CLMS

In [ ]:
# Нормализация признаков
X = ((X - X.mean(axis=0)) / X.std(axis=0)).astype("float32")

### Training model

In [ ]:
# Разделение данных на обучающее и тестовое множества
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
# mae_linear = mean_absolute_error(y_test, model.predict(X_test))
# print("MAE =", mae_linear)

### Starting Spark Session

In [ ]:
conf = pyspark.SparkConf()\
        .setAppName("bostonApp")\
        .setMaster("local[*]")

In [ ]:
spark = SparkSession\
    .builder\
    .appName("bostonApp")\
    .config(conf=conf)\
    .getOrCreate()

In [ ]:
spark

### Creating Spark DataFrame

In [ ]:
CLMS = housing.feature_names[:6] + ["MEDV"]
pdf = pd.DataFrame(data=np.c_[X_test, y_test], columns=CLMS)
pdf.head()

In [ ]:
df = spark.createDataFrame(pdf)
df.show(5)

In [ ]:
model_broadcast = spark.sparkContext.broadcast(model)

### Using UDF

In [ ]:
@udf(returnType=FloatType())
def predict_price(*row):
    model = model_broadcast.value
    return float(model.predict([row,]))

In [ ]:
CLMNS_ = CLMS.copy()
CLMNS_.remove("MEDV")
clmns = [F.col(clmn) for clmn in CLMNS_]

In [ ]:
df_predicted = df.withColumn("MEDV_PREDICT", predict_price(*clmns))
df_predicted.show(5)

In [ ]:
pdf_predicted = df_predicted.toPandas()
pdf_predicted.head()

In [ ]:
# mae_linear = mean_absolute_error(pdf_predicted["MEDV"], pdf_predicted["MEDV_PREDICT"])
# print("MAE =", mae_linear)

### Using Pandas UDF

In [ ]:
#  ~/ML/anaconda3/bin/pip install pyarrow>=0.9.0

In [ ]:
@pandas_udf("float", F.PandasUDFType.SCALAR)
def predict_price_(*cols):
    import pandas as pd
    pdf = pd.concat(cols, axis=1, ignore_index=True)
    model = model_broadcast.value
    if not isinstance(model, LinearRegression):    
        raise Exception("Not model.")
    return pd.Series(model.predict(pdf)) 

In [ ]:
CLMNS_ = CLMS.copy()
CLMNS_.remove("MEDV")
clmns = [F.col(clmn) for clmn in CLMNS_]
clmns = [clmn for clmn in CLMNS_]
clmns

In [ ]:
df_predicted = df.withColumn("MEDV_PREDICT", predict_price_(*clmns))
df_predicted.show(5)

## Model Selection

In [ ]:
from sklearn.model_selection import ParameterGrid

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge

In [ ]:
data_broadcast = spark.sparkContext.broadcast((X_train, y_train))

In [ ]:
params_grid = {
    "n_estimators": np.arange(50, 300, 50), 
    "max_depth": np.arange(5, 21, 5),
}

grid = ParameterGrid(params_grid)
list(grid)

In [ ]:
rdd_grid = spark.sparkContext.parallelize(grid, 4)
rdd_grid.glom().collect()

In [ ]:
def train(params_set):
    from sklearn.ensemble import ExtraTreesRegressor
    from sklearn.model_selection import train_test_split
    X, y = data_broadcast.value
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)
    for params in params_set:
        model = ExtraTreesRegressor(**params).fit(X_train, y_train)
        yield model.score(X_test, y_test), params

In [ ]:
rdd_grid\
    .mapPartitions(train)\
    .collect()

### Stopping Spark Session

In [ ]:
spark.stop()

## References

In [ ]:
https://spark.apache.org/docs/2.4.7/api/python/pyspark.sql.html#pyspark.sql.functions.pandas_udf
https://spark.apache.org/docs/2.4.7/sql-pyspark-pandas-with-arrow.html
https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html